In [31]:
import numpy as np
from scipy.sparse.linalg import gmres

In [46]:
def x_momentum(u_grid_dim, u_old, u_up_wall, v_old, p_old, uO, row, mu, g, delta_x, delta_y, delta_t, e):
    
    # u velocity cell coefficients
    Fe = np.zeros((u_rows - 2, u_cols))
    Fw = np.zeros((u_rows - 2, u_cols))
    Fn = np.zeros((u_rows - 2, u_cols))
    Fs = np.zeros((u_rows - 2, u_cols))
    De = np.zeros((u_rows - 2, u_cols))
    Dw = np.zeros((u_rows - 2, u_cols))
    Dn = np.zeros((u_rows - 2, u_cols))
    Ds = np.zeros((u_rows - 2, u_cols))
    aP_o = np.zeros((u_rows - 2, u_cols))
    aP = np.zeros((u_rows - 2, u_cols))
    aE = np.zeros((u_rows - 2, u_cols))
    aW = np.zeros((u_rows - 2, u_cols))
    aN = np.zeros((u_rows - 2, u_cols))
    aS = np.zeros((u_rows - 2, u_cols))
    delta_p = np.zeros((u_rows - 2, u_cols))
    d_u = np.zeros((u_rows - 2, u_cols))
    # contrcuting a zero matrix
    zero = np.zeros((u_rows - 2, u_cols))
    
    # calculating mass flow rates at staggered cell centres:
    Fe[:, -1] = 0.0
    Fe[:, :-1] = row / 2.0 * (u_values[1:-1, :-1] + u_values[1:-1, 1:])

    Fw[:, 0] = 0.0
    Fw[:, 1:] = row / 2.0 * (u_values[1:-1, :-1] + u_values[1:-1, 1:])

    Fn[0, :] = 0 
    Fn[1:, :] = row / 2.0 * (v_values[1:-1, :-1] + v_values[1:-1, 1:])

    Fs[-1, :] = 0
    Fs[0:-1, :] = row / 2.0 * (v_values[1:-1, :-1] + v_values[1:-1, 1:])
    
    # calculating diffusion fluxes at staggered cell centres:
    De[:, -1] = 0.0
    De[:, :-1] = mu / delta_x
    Dw[:, 0] = 0.0
    Dw[:, 1:] = mu / delta_x
    Dn[0, :] = 0.0
    Dn[1:, :] = mu / delta_y
    Ds[-1, :] = 0.0
    Ds[0:-1, :] = mu / delta_y
    
    # pressure source term: 
    delta_p[:, :] = (p_values[1: -1, :- 1] - p_values[1: -1, 1:]) * delta_y
    # body force term + pressure source term:
    b = delta_p + row * g
    
    # calculating neighbouring coeff. for a staggered cell centres:
    aE = (De + np.fmax(-Fe, zero)) * delta_y
    aW = (Dw + np.fmax(Fw, zero)) * delta_y
    aN = (Dn + np.fmax(-Fn, zero)) * delta_x
    aS = (Ds + np.fmax(Fs, zero)) * delta_x
    aP_o[:, :] = (row * delta_x * delta_y) / delta_t
    
    # calculating central coeff. of staggered cells
    aP = aP_o + np.fmax(Fe, zero) * delta_y + np.fmax(-Fw, zero) * delta_y + np.fmax(Fn, zero) * delta_x + \
         np.fmax(-Fs, zero) * delta_x + De * delta_y + Dw * delta_y + Dn * delta_x + Ds * delta_x
        
    n_u = 0
    # solving for u - velocity in the grid untill convergence
    while True:
        n_u += 1
        '''
        Fe[:, -1] = 0.0
        Fe[:, :-1] = row / 2.0 * (u_values[1:-1, :-1] + u_values[1:-1, 1:])

        Fw[:, 0] = 0.0
        Fw[:, 1:] = row / 2.0 * (u_values[1:-1, :-1] + u_values[1:-1, 1:])

        Fn[0, :] = 0 
        Fn[1:, :] = row / 2.0 * (v_values[1:-1, :-1] + v_values[1:-1, 1:])

        Fs[-1, :] = 0
        Fs[0:-1, :] = row / 2.0 * (v_values[1:-1, :-1] + v_values[1:-1, 1:])

        De[:, -1] = 0.0
        De[:, :-1] = mu / delta_x
        Dw[:, 0] = 0.0
        Dw[:, 1:] = mu / delta_x
        Dn[0, :] = 0.0
        Dn[1:, :] = mu / delta_y
        Ds[-1, :] = 0.0
        Ds[0:-1, :] = mu / delta_y

        delta_p[:, :] = (p_values[1: -1, :- 1] - p_values[1: -1, 1:]) * delta_y
        b = delta_p + row * g

        aE[:, :] = (De[:, :] + np.fmax(-Fe[:, :], zero[:, :])) * delta_y
        aW[:, :] = (Dw[:, :] + np.fmax(Fw[:, :], zero[:, :])) * delta_y
        aN[:, :] = (Dn[:, :] + np.fmax(-Fn[:, :], zero[:, :])) * delta_x
        aS[:, :] = (Ds[:, :] + np.fmax(Fs[:, :], zero[:, :])) * delta_x
        aP_o[:, :] = (row * delta_x * delta_y) / delta_t

        aP[:, :] = aP_o[:, :] + np.fmax(Fe[:, :], zero[:, :]) * delta_y + np.fmax(-Fw[:, :], zero[:, :]) * delta_y + np.fmax(Fn[:, :], zero[:, :]) * delta_x + \
             np.fmax(-Fs[:, :], zero[:, :]) * delta_x + De[:, :] * delta_y + Dw[:, :] * delta_y + Dn[:, :] * delta_x + Ds[:, :] * delta_x

        print(aP, '\n')
        '''
        # Re-arranging the equations in the form: -aNUN + aPUP - aSUS = u_source, where u_source = aEUE + aWUW + b
        # After re-arranging in the above form, equations are solved using TDMA scheme for every North-South direction in the grid,
        # sweeping from West to East.
        u_vals = u_values.copy()
        u_source = np.zeros(u_rows -2)

        for col in range(0, u_cols):

            tdma = np.diag(aP[:, col], k=0) + np.diag(-aN[1:, col], k=-1) + np.diag(-aS[:-1, col], k=1)

            if col == 0:
                u_source[:] = b[:, col] + aE[:, col] * u_values[1:-1, col + 1] + aP_o[:, col] * uO[1:-1, col]
            elif col == u_cols - 1:
                u_source[:] = b[:, col] + aW[:, col] * u_values[1:-1, col - 1] + aP_o[:, col] * uO[1:-1, col]
            else:
                u_source[:] = b[:, col] + aE[:, col] * u_values[1:-1, col + 1] + aW[:, col] * u_values[1:-1, col - 1] + aP_o[:, col] * uO[1:-1, col]

            # implementing BC: u = 0 at left and right wall
            if col == 0 or col == u_cols - 1:
                tdma[:] = 0.0

                for i in range(u_rows - 2):
                    for j in range(u_rows - 2):
                        if i == j:
                            tdma[i, j] = 1.0

                u_source[:] = 0.0
            else:
                tdma[0, :] = 0.0
                tdma[0,0] = 1.0
                u_source[0] = u_up_wall # from BC on up wall........

            # solving for u momentum:
            u_solution = gmres(tdma, u_source)[0]

            # updating u values grid:
            u_values[1:-1, col] = u_solution[:]

            #BC: upper wall is moving:
            if col == 0 or col == u_cols - 1:
                u_values[0, col] = 2.0
            else:
                u_values[0, col] = u_solution[0]

            #BC: lower wall is stationary, u = 0
            u_values[-1, col] =  - u_solution[-1]

        residue = abs(u_vals[1: -1, 1: -1] - u_values[1: -1, 1: -1])
        #print('************ Residue **************', '\n')
        #print(np.sum(residue), '\n')
        if np.sum(residue) < e or n_u > 1e04:
            break

    return((u_values, aP))

In [52]:
def y_momentum(v_grid_dim, v_values, u_values, p_values, vO, row, mu, g, delta_x, delta_y, delta_t, e):
    
    v_rows, v_cols = v_grid_dim
    
    # v velocity cell coefficients
    Fe_ = np.zeros((v_rows, v_cols - 2))
    Fw_ = np.zeros((v_rows, v_cols - 2))
    Fn_ = np.zeros((v_rows, v_cols - 2))
    Fs_ = np.zeros((v_rows, v_cols - 2))
    De_ = np.zeros((v_rows, v_cols - 2))
    Dw_ = np.zeros((v_rows, v_cols - 2))
    Dn_ = np.zeros((v_rows, v_cols - 2))
    Ds_ = np.zeros((v_rows, v_cols - 2))
    aP_o_ = np.zeros((v_rows, v_cols - 2))
    aP_ = np.zeros((v_rows, v_cols - 2))
    aE_ = np.zeros((v_rows, v_cols - 2))
    aN_ = np.zeros((v_rows, v_cols - 2))
    aS_ = np.zeros((v_rows, v_cols - 2))
    aW_ = np.zeros((v_rows, v_cols - 2))
    delta_p_ = np.zeros((v_rows, v_cols - 2))
    d_v = np.zeros((v_rows, v_cols - 2))
    # contrcuting a zero matrix
    zero_ = np.zeros((v_rows, v_cols - 2))
    
    # calculating mass flow rates at staggered cell centres:
    Fs_[-1, :] = 0.0
    Fs_[:-1, :] = row / 2.0 * (v_values[:-1, 1:-1] + v_values[1:, 1:-1])

    Fn_[0, :] = 0.0
    Fn_[1:, :] = row / 2.0 * (v_values[:-1, 1:-1] + v_values[1:, 1:-1])

    Fw_[:, 0] = 0 
    Fw_[:, 1:] = row / 2.0 * (u_values[:-1, 1:-1] + u_values[1:, 1:-1])

    Fe_[:, -1] = 0
    Fe_[:, :-1] = row / 2.0 * (u_values[:-1, 1:-1] + u_values[1:, 1:-1])
    
    # calculating diffusion fluxes at staggered cell centres:
    De_[:, -1] = 0.0
    De_[:, 0: -1] = mu / delta_x
    Dw_[:, 0] = 0.0
    Dw_[:, 1:] = mu / delta_x
    Dn_[0, :] = 0.0
    Dn_[1:, :] = mu / delta_y
    Ds_[-1, :] = 0.0
    Ds_[:-1, :] = mu / delta_y
    
    # pressure source term: 
    delta_p_ = (p_values[:-1, 1: -1] - p_values[1:, 1: -1]) * delta_y
    # body force term + pressure source term:
    b_ = delta_p_ + row * g
    
    # calculating neighbouring coeff. for a staggered cell centres:
    aE_ = (De_ + np.fmax(-Fe_, zero_)) * delta_y
    aW_ = (Dw_ + np.fmax(Fw_, zero_)) * delta_y
    aN_ = (Dn_ + np.fmax(-Fn_, zero_)) * delta_x
    aS_ = (Ds_ + np.fmax(Fs_, zero_)) * delta_x
    aP_o_[:, :] = (row * delta_x * delta_y) / delta_t
    
    # calculating central coeff. of staggered cells
    aP_ = aP_o_ + np.fmax(Fe_, zero_) * delta_y + np.fmax(-Fw_, zero_) * delta_y + np.fmax(Fn_, zero_) * delta_x + \
          np.fmax(-Fs_, zero_) * delta_x + De_ * delta_y + Dw_ * delta_y + Dn_ * delta_x + Ds_ * delta_x
        
    # solving for v - velocity in the grid untill convergence
    n_v = 0
    while True:

        n_v += 1
        '''
        Fs_[-1, :] = 0.0
        Fs_[:-1, :] = row / 2.0 * (v_values[:-1, 1:-1] + v_values[1:, 1:-1])

        Fn_[0, :] = 0.0
        Fn_[1:, :] = row / 2.0 * (v_values[:-1, 1:-1] + v_values[1:, 1:-1])

        Fw_[:, 0] = 0 
        Fw_[:, 1:] = row / 2.0 * (u_values[:-1, 1:-1] + u_values[1:, 1:-1])

        Fe_[:, -1] = 0
        Fe_[:, :-1] = row / 2.0 * (u_values[:-1, 1:-1] + u_values[1:, 1:-1])

        De_[:, -1] = 0.0
        De_[:, 0: -1] = mu / delta_x
        Dw_[:, 0] = 0.0
        Dw_[:, 1:] = mu / delta_x
        Dn_[0, :] = 0.0
        Dn_[1:, :] = mu / delta_y
        Ds_[-1, :] = 0.0
        Ds_[:-1, :] = mu / delta_y

        delta_p_[:, :] = (p_values[:-1, 1: -1] - p_values[1:, 1: -1]) * delta_y
        b_ = delta_p_ + row * g

        aE_[:, :] = (De_[:, :] + np.fmax(-Fe_[:, :], zero_[:, :])) * delta_y
        aW_[:, :] = (Dw_[:, :]+ np.fmax(Fw_[:, :], zero_[:, :])) * delta_y
        aN_[:, :] = (Dn_[:, :] + np.fmax(-Fn_[:, :], zero_[:, :])) * delta_x
        aS_[:, :] = (Ds_[:, :] + np.fmax(Fs_[:, :], zero_[:, :])) * delta_x
        aP_o_[:, :] = (row * delta_x * delta_y) / delta_t

        aP_[:, :] = aP_o_[:, :] + np.fmax(Fe_[:, :], zero_[:, :]) * delta_y + np.fmax(-Fw_[:, :], zero_[:, :]) * delta_y + np.fmax(Fn_[:, :], zero_[:, :]) * delta_x + \
              np.fmax(-Fs_[:, :], zero_[:, :]) * delta_x + De_[:, :] * delta_y + Dw_[:, :] * delta_y + Dn_[:, :] * delta_x + Ds_[:, :] * delta_x
        '''
        # Re-arranging the equations in the form: -aEUE + aPUP - aWUW = u_source, where u_source = aNUN + aSUS + b
        # After re-arranging in the above form, equations are solved using TDMA scheme for every North-South direction in the grid,
        # sweeping from West to East.
        v_vals = v_values.copy()
        v_source = np.zeros(v_cols - 2)

        for rows in np.negative(np.arange(1, v_rows)):

            tdma_ = np.diag(aP_[rows, :], k=0) + np.diag(-aE_[rows, :-1], k=1) + np.diag(-aW_[rows, 1:], k=-1)

            if rows == 0:
                v_source[:] = b_[rows, :] + aN_[rows, :] * v_values[rows + 1, 1:-1] + aP_o_[rows, :] * vO[rows, 1:-1]
            elif rows == v_rows - 1:
                v_source[:] = b_[rows, :] + aS_[rows, :] * v_values[rows - 1, 1:-1] + aP_o_[rows, :] * vO[rows, 1:-1]
            else:
                v_source[:] = b_[rows, :] + aN_[rows, :] * v_values[rows + 1, 1:-1] + aS_[rows, :] * v_values[rows - 1, 1:-1] + aP_o_[rows, :] * vO[rows, 1:-1] 


            # BC v = 0 at up and down wall:
            if rows == -1 or rows == -v_rows:

                tdma_[:] = 0.0

                for i in range(v_cols - 2):
                    for j in range(v_cols - 2):
                        if i == j:
                            tdma_[i, j] = 1.0

                v_source[:] = 0.0

            # solving for v momentum:
            v_solution = gmres(tdma_, v_source)[0]

            # updating v values grid:
            v_values[rows, 1: -1] = v_solution[:]

            #BC: left and right wall are stationary:
            v_values[rows, 0] = - v_solution[0]
            v_values[rows, -1] = - v_solution[-1]

        residue = abs(v_vals[1: -1, 1: -1] - v_values[1: -1, 1: -1])
        #print('************ Residue **************', '\n')
        #print(np.sum(residue), '\n')
        if np.sum(residue) < e or n_v > 1e04:
            break

    return((v_values, aP_))

In [65]:
def pressure(p_grid_dim, aP_u, aP_v, u_values, v_values, row, mu, delta_x, delta_y, delta_t):
    
    p_rows, p_cols = p_grid_dim
    
    # pressure coefficients:
    aE_p = np.zeros((p_rows - 2, p_cols - 2))
    aW_p = np.zeros((p_rows - 2, p_cols - 2))
    aS_p = np.zeros((p_rows - 2, p_cols - 2))
    aN_p = np.zeros((p_rows - 2, p_cols - 2))
    aP_p = np.zeros((p_rows - 2, p_cols - 2))
    b_p_we = np.zeros((p_rows - 2, p_cols - 2))
    b_p_sn = np.zeros((p_rows - 2, p_cols - 2))
    b_p = np.zeros((p_rows - 2, p_cols - 2))
    
    # calculating d coeff. for pressure correction
    d_u = delta_y / aP_u
    d_v = delta_x / aP_v
    
    aE_p[:, : -1] = row * d_u[:, 1: -1] * delta_y
    aW_p[:, 1:] = row * d_u[:, 1: -1] * delta_y
    aS_p[: -1, :] = row * d_v[1: -1, :] * delta_x
    aN_p[1:, :] = row * d_v[1: -1, :] * delta_x
    
    
    aP_p[:, :] = aE_p[:, :] + aW_p[:, :] + aN_p[:, :] + aS_p[:, :]
    
    # pressure source term: (convergence check will be performed on this variable)
    b_p_we[:, :] = row * delta_y * (u_values[1: -1, :-1] - u_values[1: -1, 1:])
    b_p_sn[:, :] = row * delta_x * (v_values[1:, 1: -1] - v_values[: -1, 1: -1])
    b_p[:, :] = b_p_we[:, :] + b_p_sn[:, :]
    
    # making a copy of old p values
    p_ = np.zeros((p_grid_dim))
    
    # solving for pressure in the grid until convergence:
    n_p = 0
    while True:

        n_p += 1

        p_source = np.zeros(p_rows - 2)
        p_vals = p_.copy()

        for cols in range(0, p_cols):

            if cols >= 1 and cols <= p_cols - 2:
                
                # constrcting p_tdma
                p_tdma = np.diag(aP_p[:, cols - 1], k=0) + np.diag(-aN_p[1:, cols - 1], k=-1) + np.diag(-aS_p[: -1, cols - 1], k=1)

                if cols == 1:
                    p_source[:] = b_p[:, cols - 1] + aE_p[:, cols - 1] * p_[1: -1, cols + 1]
                elif cols == p_cols - 2:
                    p_source[:] = b_p[:, cols - 1] + aW_p[:, cols - 1] * p_[1: -1, cols - 1]
                else:
                    p_source[:] = b_p[:, cols - 1] + aE_p[:, cols - 1] * p_[1: -1, cols + 1] + aW_p[:, cols - 1] * p_[1: -1, cols - 1]
                
                # from BC:
                if cols == p_cols - 2:
                    p_tdma[1, :] = 0.0
                    #p_tdma[0, 0] = 1.0
                    p_tdma[1, 1] = 1.0
                    #p_tdma[2, 2] = 1.0
                    p_source[1] = 0.0 # pressure is zero at one cell near right wall

                # solving for pressure:
                p_solution = gmres(p_tdma, p_source)[0]

                # from BC:
                if cols == 1:
                    p_[1: -1, cols - 1] = p_solution[:]
                elif cols == p_cols - 2:
                    p_[1: -1, cols + 1] = p_solution[:]

                p_[1: -1, cols] = p_solution[:]

                p_[0, cols] = p_solution[0]
                p_[-1, cols] = p_solution[-1]

        residue = abs(p_vals[1: -1, 1: -1] - p_[1: -1, 1: -1])
        #print('************ Residue **************', '\n')
        #print(np.sum(residue), '\n')
        if np.sum(residue)< 0.001 or n_p > 1e04:
            break
    
    return((p_, b_p, d_u, d_v))

In [63]:
def pressure_correction(u_n, u_old, v_n, v_old, p_n, p_old, alpha_u, alpha_v, alpha_p, d_u, d_v):
    
    # correcting u:
    u_cor = u_n.copy()
    u_cor[2: -1, 1: -1] += d_u[1:, 1: -1] * (p_n[2: -1 , 1: -2] - p_n[2: -1, 2: -1])
    u_cor[-1, :] = - u_cor[-2, :] # from down wall BC.

    u_new = u_cor.copy()
    u_new[2: -1, 1: -1] = alpha_u * u_cor[2: -1, 1: -1] + (1.0 - alpha_u) * u_old[2: -1, 1: -1]
    u_new[-1, :] = - u_new[-2, :] # from down wall BC.

    # correcting v:
    v_cor = v_n.copy()
    v_cor[1: -1, 1: -1] += d_v[1: -1, 0: ] * (p_old[1: -2 , 1: -1] - p_old[2: -1, 1: -1])
    # from left and right wall BC:
    v_cor[:, 0] = - v_cor[:, 1]
    v_cor[:, -1] = - v_cor[:, -2]

    v_new = v_cor.copy()
    v_new[1: -1, 1: -1]  = alpha_v * v_cor[1: -1, 1: -1]  + (1.0 - alpha_v) * v_old[1: -1, 1: -1] 
    # from left and right wall BC:
    v_new[:, 0] = - v_new[:, 1]
    v_new[:, -1] = - v_new[:, -2]

    # correcting p:
    p_new = np.zeros((p_grid_dim))
    p_new[1: -1, 1: -1] = p_old[1: -1, 1: -1] + alpha_p * p_n[1: -1, 1: -1]
    # satisfying BC for pressure:
    p_new[1: -1, 0] = p_new[1: -1, 1]
    p_new[1: -1, -1] = p_new[1: -1, -2]
    p_new[0, 1: -1] = p_new[1, 1: -1]
    p_new[-1, 1: -1] = p_new[-2, 1: -1]
    
    return((u_new, v_new, p_new))

In [50]:
def check_convergence(e, b_p, u_cor, u_old, v_cor, v_old, p_cor, p_old):
    
    convergence = False
    
    p_dim = p_cor.shape
    p_rows, p_cols = p_dim
    
    # checking convergence:

    #residue of u:
    residue_u = abs(u_cor[1: -1, 1: -1] - u_old[1: -1, 1: -1])
    #residue of v:
    residue_v = abs(v_cor[1: -1, 1: -1] - v_old[1: -1, 1: -1])
    #residue of p:
    residue_p = abs(p_cor[1: -1, 1: -1] - p_old[1: -1, 1: -1])

    print('*************** u- residue ****************', '\n')
    print(np.sum(residue_u), '\n')
    print('*************** v- residue ****************', '\n')
    print(np.sum(residue_v), '\n')
    print('*************** p- residue ****************', '\n')
    print(np.sum(residue_p), '\n')
    print('\n')

    if b_p[b_p < e].shape == ((p_rows - 2) * (p_cols - 2),):
        print('################ Convergence #############', '\n')
        print('!!!!!!!!!! Time = ', t, '!!!!!!!!!!!', '\n')
        print('*************** u-velocity solution ****************', '\n')
        print(u_cor, '\n')
        print('*************** v-velocity solution ****************', '\n')
        print(v_cor, '\n')
        print('*************** pressure solution ****************', '\n')
        print(p_cor, '\n')
        
        convergence = True
        
    return(convergence)

In [72]:
# given B.Cs.
left = 'stationary_wall'
right = 'stationary_wall'
down = 'stationary_wall'
up = 'moving_wall'
u_up_wall = 1.0 # m/sec

# physical properties:
length = 1.0  # m
row = 1.0     # kg/m3
Re = 400.0
mu = row * u_up_wall * length / Re
g = 0.0

# under-relaxation parameters:
alpha_u = 1.
alpha_v = 1.
alpha_p = 0.01

# time:
t_step = 0.0
delta_t = 0.1
t = 1e03

# total inerations allowed in one time step:
total_iter = 1e03
n_iter = 0

# error
e = 0.01

# setting up the grid --- ghost cell method has been adopted while setting up the grid ----
u_grid_dim = (5, 4)
u_grid = np.zeros(u_grid_dim)
u_rows, u_cols = u_grid_dim

v_grid_dim = (u_grid_dim[1], u_grid_dim[0])
v_grid = np.zeros(v_grid_dim)
v_rows, v_cols = v_grid_dim

p_grid_dim = (u_grid_dim[0], u_grid_dim[0])
p_grid = np.zeros(p_grid_dim)
p_rows, p_cols = p_grid_dim

no_of_cv = (u_rows + u_cols - 3) / 2
delta_x = length / no_of_cv
delta_y = length / no_of_cv
rows = u_rows + u_cols
cols = u_rows + u_cols

grid = np.zeros((rows, cols))

# initializing the values in grid

# u velocity
u_grid[0, 0] = 2.0
u_grid[0, -1] = 2.0
u_grid[0:2, 1: -1] = u_up_wall

# v velocity
v_grid[:] = 0.0

# p values
p_grid[:] = 0.0

# extracting values of u - velocity from main grid
u_values = u_grid[:, :].copy()
uO = u_grid[:, :].copy()
# extracting values of u - velocity from main grid
v_values = v_grid[:, :].copy()
vO = v_grid[:, :].copy()
# extracting values of p from main grid
p_values = p_grid[:, :].copy()

while True:
    
    print('############ Iteration ', n_iter, ' ############', '\n')
    
    # copying values of u and v from last iteration:
    u_old = u_values.copy()
    v_old = v_values.copy()
    p_old = p_values.copy()

    u_new, aP_u = x_momentum(u_grid_dim, u_old, u_up_wall, v_old, p_old, uO, row, mu, g, delta_x, delta_y, delta_t, e)

    v_new, aP_v = y_momentum(v_grid_dim, v_old, u_old, p_old, vO, row, mu, g, delta_x, delta_y, delta_t, e)

    p_new, b_p, d_u, d_v = pressure(p_grid_dim, aP_u, aP_v, u_new, v_new, row, mu, delta_x, delta_y, delta_t)

    u_cor, v_cor, p_cor = pressure_correction(u_new, u_old, v_new, v_old, p_new, p_old, alpha_u, alpha_v, alpha_p, d_u, d_v) 

    convergence = check_convergence(e, b_p, u_cor, u_old, v_cor, v_old, p_cor, p_old)

    if convergence is True:
        break
    elif n_iter == total_iter:
        print('############ No Convergence #########')
        break
    else:
        u_values[:, :] = u_cor[:, :]
        v_values[:, :] = v_cor[:, :]
        p_values[:, :] = p_cor[:, :]

    n_iter += 1

############ Iteration  0  ############ 

*************** u- residue **************** 

0.823722554762 

*************** v- residue **************** 

0.0 

*************** p- residue **************** 

0.112837267954 



############ Iteration  1  ############ 

*************** u- residue **************** 

0.00878211852636 

*************** v- residue **************** 

0.0111376224227 

*************** p- residue **************** 

0.115843812066 



############ Iteration  2  ############ 

*************** u- residue **************** 

0.0024989251806 

*************** v- residue **************** 

0.0223059140196 

*************** p- residue **************** 

0.115406540617 



############ Iteration  3  ############ 

*************** u- residue **************** 

0.00231788771977 

*************** v- residue **************** 

0.0335271018257 

*************** p- residue **************** 

0.115008992625 



############ Iteration  4  ############ 

*************** u- residue **

0.00401724158471 

*************** v- residue **************** 

0.590450149148 

*************** p- residue **************** 

0.101824709491 



############ Iteration  49  ############ 

*************** u- residue **************** 

0.00405589822801 

*************** v- residue **************** 

0.603913470452 

*************** p- residue **************** 

0.101628545639 



############ Iteration  50  ############ 

*************** u- residue **************** 

0.00409447485865 

*************** v- residue **************** 

0.617421282234 

*************** p- residue **************** 

0.101436231421 



############ Iteration  51  ############ 

*************** u- residue **************** 

0.00413297202648 

*************** v- residue **************** 

0.630973416383 

*************** p- residue **************** 

0.101247748009 



############ Iteration  52  ############ 

*************** u- residue **************** 

0.00422238144858 

*************** v- residue **********


*************** v- residue **************** 

1.53419085964 

*************** p- residue **************** 

0.09630072847 



############ Iteration  111  ############ 

*************** u- residue **************** 

0.0063137693621 

*************** v- residue **************** 

1.55115200629 

*************** p- residue **************** 

0.0963120394434 



############ Iteration  112  ############ 

*************** u- residue **************** 

0.00634818074917 

*************** v- residue **************** 

1.56816827395 

*************** p- residue **************** 

0.0963263311483 



############ Iteration  113  ############ 

*************** u- residue **************** 

0.006382553044 

*************** v- residue **************** 

1.58523974618 

*************** p- residue **************** 

0.0963436009242 



############ Iteration  114  ############ 

*************** u- residue **************** 

0.00646918644518 

*************** v- residue **************** 

1.60236650

0.110854263835 



############ Iteration  165  ############ 

*************** u- residue **************** 

0.00811748290214 

*************** v- residue **************** 

2.55032083404 

*************** p- residue **************** 

0.111329695793 



############ Iteration  166  ############ 

*************** u- residue **************** 

0.00814997923908 

*************** v- residue **************** 

2.57035293879 

*************** p- residue **************** 

0.111808273822 



############ Iteration  167  ############ 

*************** u- residue **************** 

0.00818245277451 

*************** v- residue **************** 

2.59044610108 

*************** p- residue **************** 

0.1122900167 



############ Iteration  168  ############ 

*************** u- residue **************** 

0.00819699694454 

*************** v- residue **************** 

2.61046611363 

*************** p- residue **************** 

0.112770863319 



############ Iteration  169  ##########

0.13573617579 



############ Iteration  204  ############ 

*************** u- residue **************** 

0.00931611102854 

*************** v- residue **************** 

3.37118444713 

*************** p- residue **************** 

0.136617314645 



############ Iteration  205  ############ 

*************** u- residue **************** 

0.00934608006062 

*************** v- residue **************** 

3.39346887375 

*************** p- residue **************** 

0.137502237301 



############ Iteration  206  ############ 

*************** u- residue **************** 

0.00937564605109 

*************** v- residue **************** 

3.41581809052 

*************** p- residue **************** 

0.13839143401 



############ Iteration  207  ############ 

*************** u- residue **************** 

0.00940520618251 

*************** v- residue **************** 

3.43823178567 

*************** p- residue **************** 

0.139284969656 



############ Iteration  208  ##########

...

11.2858412399 



############ Iteration  737  ############ 

*************** u- residue **************** 

0.00762516636137 

*************** v- residue **************** 

45.2280217406 

*************** p- residue **************** 

11.4063971034 



############ Iteration  738  ############ 

*************** u- residue **************** 

0.0081399813723 

*************** v- residue **************** 

45.5309981663 

*************** p- residue **************** 

11.5278804438 



############ Iteration  739  ############ 

*************** u- residue **************** 

0.00782692736319 

*************** v- residue **************** 

45.8336861324 

*************** p- residue **************** 

11.6509139779 



############ Iteration  740  ############ 

*************** u- residue **************** 

0.00812293144862 

*************** v- residue **************** 

46.1395168811 

*************** p- residue **************** 

11.7750617098 



############ Iteration  741  ############ 





############ Iteration  778  ############ 

*************** u- residue **************** 

0.0114073181168 

*************** v- residue **************** 

59.0898337887 

*************** p- residue **************** 

17.6415390605 



############ Iteration  779  ############ 

*************** u- residue **************** 

0.0115190556695 

*************** v- residue **************** 

59.4690625021 

*************** p- residue **************** 

17.8310715471 



############ Iteration  780  ############ 

*************** u- residue **************** 

0.0116677087667 

*************** v- residue **************** 

59.8504235022 

*************** p- residue **************** 

18.022653567 



############ Iteration  781  ############ 

*************** u- residue **************** 

0.0117921234053 

*************** v- residue **************** 

60.2339086752 

*************** p- residue **************** 

18.2163150232 



############ Iteration  782  ############ 

*************** u-

30.4725706168 



############ Iteration  830  ############ 

*************** u- residue **************** 

0.015652341117 

*************** v- residue **************** 

81.9120196393 

*************** p- residue **************** 

30.801956085 



############ Iteration  831  ############ 

*************** u- residue **************** 

0.0156752947541 

*************** v- residue **************** 

82.4192463837 

*************** p- residue **************** 

31.1349450402 



############ Iteration  832  ############ 

*************** u- residue **************** 

0.0156946600045 

*************** v- residue **************** 

82.9293090055 

*************** p- residue **************** 

31.4715757054 



############ Iteration  833  ############ 

*************** u- residue **************** 

0.0157135501295 

*************** v- residue **************** 

83.4422376599 

*************** p- residue **************** 

31.8118892252 



############ Iteration  834  ############ 

****

*************** u- residue **************** 

0.0149614687387 

*************** v- residue **************** 

103.324748223 

*************** p- residue **************** 

46.4026111687 



############ Iteration  869  ############ 

*************** u- residue **************** 

0.0149047225085 

*************** v- residue **************** 

103.951826286 

*************** p- residue **************** 

46.9089569474 



############ Iteration  870  ############ 

*************** u- residue **************** 

0.0148295680876 

*************** v- residue **************** 

104.582412161 

*************** p- residue **************** 

47.4208826073 



############ Iteration  871  ############ 

*************** u- residue **************** 

0.0147907978109 

*************** v- residue **************** 

105.21652789 

*************** p- residue **************** 

47.93844058 



############ Iteration  872  ############ 

*************** u- residue **************** 

0.0147270630004 

***

############ Iteration  904  ############ 

*************** u- residue **************** 

0.0142299701229 

*************** v- residue **************** 

128.253659331 

*************** p- residue **************** 

68.6327653824 



############ Iteration  905  ############ 

*************** u- residue **************** 

0.0141167558081 

*************** v- residue **************** 

129.020255896 

*************** p- residue **************** 

69.3846874103 



############ Iteration  906  ############ 

*************** u- residue **************** 

0.0139939692532 

*************** v- residue **************** 

129.791163204 

*************** p- residue **************** 

70.144898888 



############ Iteration  907  ############ 

*************** u- residue **************** 

0.013864801783 

*************** v- residue **************** 

130.566315632 

*************** p- residue **************** 

70.91348478 



############ Iteration  908  ############ 

*************** u- resid